In [57]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hw-texts/baseline_submit.csv
/kaggle/input/hw-texts/train.csv
/kaggle/input/hw-texts/test.csv


In [58]:
#from tqdm.auto import tqdm, trange

In [59]:
! pip install pymorphy2[fast] rnnmorph pymystem3 nltk razdel

In [60]:
train = pd.read_csv('/kaggle/input/hw-texts/train.csv', sep=';')
test = pd.read_csv('/kaggle/input/hw-texts/test.csv', sep=';')
submission = pd.read_csv('/kaggle/input/hw-texts/baseline_submit.csv')

In [61]:
'train: ', train.shape, 'test: ', test.shape #оцениваем размер

('train: ', (78726, 3), 'test: ', (33741, 2))

In [62]:
submission.head()  

,id,label
0,0,2
1,1,2
2,2,2
3,3,2
4,4,1


In [63]:
train.head()

,id,text,label
0,0,Я не знаю какая Ты в жизни в постели в быту...,1
1,1,Херсон рулит есть д для жизни я п 32 068 160 9...,1
2,2,"решил поглумиться, 15000 итераций 0.001 лр",2
3,3,М.48 ДЛЯ ЛЮДЫ! СМС 093.087.16.85,1
4,4,М 33 позн с д/ж 093 76 46 933 Киев,1


In [64]:
# Отобразим количество текста из разных чатов
train.label.value_counts(normalize=True)
#"1" - чат знакомств
#"2" - чат про ml
#"0" - чат о python

1    0.446079
2    0.361901
0    0.192020
Name: label, dtype: float64

In [65]:
train[train['label']==1].head(10)

,id,text,label
0,0,Я не знаю какая Ты в жизни в постели в быту...,1
1,1,Херсон рулит есть д для жизни я п 32 068 160 9...,1
3,3,М.48 ДЛЯ ЛЮДЫ! СМС 093.087.16.85,1
4,4,М 33 позн с д/ж 093 76 46 933 Киев,1
5,5,парень скучает девушки приселайте прилольние м...,1
13,13,Ищу девушку для серьёзных отношений 0633096850...,1
15,15,Приглашу в гости ухоженую Д-Ж 27-40 лет я паре...,1
16,16,симп п покатаю по ноч городу сем пару дев+шамп...,1
18,18,любимый! я так скучаю!!!!!! Ирина,1
21,21,ПРИВЕТ МОЙ РОДНУЛЬКА И РЕБЯТА.КОТ ДНЕМ СИ НЕ Л...,1


In [66]:
train[train['label']==2].head(10)

,id,text,label
2,2,"решил поглумиться, 15000 итераций 0.001 лр",2
8,8,"да и все остальные ""сглаживаются"" спомощью pri...",2
9,9,"вангую, что выйдет чуть больше 1700 в месяц чи...",2
10,10,Надо рисоватт ключи на бумаге? Получателя фсб ...,2
11,11,"Возможно, но пишут simple и бесчисленные приме...",2
12,12,"ну я к тому, что а в истории с Платовым треш к...",2
14,14,"кондорсе примерно тоже самое, вид сбоку\nя про...",2
17,17,а скок ты рост/вес?,2
19,19,я бы даже без обучения так бы сделал,2
20,20,"Божечки, второй вечер подряд обсуждение гендер...",2


In [67]:
train[train['label']==0].head(10)

,id,text,label
6,6,"argparse прекрасно делает все то, что мне от н...",0
7,7,Напиши в поиске винды «Переименовать компьютер».,0
47,47,уровень вида “дайте две недели и я буду писать...,0
51,51,"Блин, офигенно. Надо бы найти, может ли она по...",0
53,53,Всем привет! Как поймать исключение другого по...,0
55,55,Раньше нагрузки небыло - она справлялась. Щас ...,0
60,60,"вообще то можно что угодно сделать, если прави...",0
62,62,"поставил 18.04, в родном идет только python 3.6",0
64,64,"Мне нужно рисовать, не диаграммы по стандарту....",0
66,66,"тогда, это второе, а там правильный 66)",0


#  1. Предобработка текста

In [68]:
import re
import pandas as pd
import fasttext
from sklearn.model_selection import train_test_split

from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem  #Библиотека морфологии от Яндекса
from string import punctuation
mystem = Mystem()

russian_stopwords = stopwords.words('russian')

import string
def punctuation(text):
    return ''.join([ch if ch not in string.punctuation else ' ' for ch in text])
def numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])
def multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)
def t_lower(text):
    return text.lower()

def lemmatize_text(text):
    tokens = mystem.lemmatize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token!=' ']
    text = ' '.join(tokens)
    return text

In [69]:
train['text'] = [multiple_spaces(numbers(punctuation(t_lower(text)))) for text in train['text']]

In [70]:
train

,id,text,label
0,0,я не знаю какая ты в жизни в постели в быту но...,1
1,1,херсон рулит есть д для жизни я п,1
2,2,решил поглумиться итераций лр,2
3,3,м для люды смс,1
4,4,м позн с д ж киев,1
...,...,...,...
78721,78721,у меня наверно говенные авиалинии nно вот все ...,2
78722,78722,он по крайне мере бесплатный имеет нормальный ...,0
78723,78723,я уточню у грефа ладно ладно,2
78724,78724,да пока так и делаю но у ребят все быстро меня...,2


# Стемминг

In [71]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer('russian')

In [72]:
russian_stopwords = stopwords.words('russian')
russian_stopwords

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

Произведем токенизацию по словам

In [73]:
from nltk import word_tokenize
stemmed_t = []
for text in train['text']:
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = ' '. join(stemmed_tokens)
    stemmed_t.append(text)

train['text_stem']=stemmed_t

In [74]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [75]:
def remove_stop_words(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token!=' ']
    return ' '.join(tokens)

In [76]:
del_t=[]
for text in train['text']:
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token !=' ']
    text = ' '.join(tokens)
    del_t.append(text)
train['text_del']=del_t

In [77]:
train

,id,text,label,text_stem,text_del
0,0,я не знаю какая ты в жизни в постели в быту но...,1,зна жизн постел быт точн зна душ тво любл,знаю жизни постели быту точно знаю душу твою л...
1,1,херсон рулит есть д для жизни я п,1,херсон рул д жизн п,херсон рулит д жизни п
2,2,решил поглумиться итераций лр,2,реш поглум итерац лр,решил поглумиться итераций лр
3,3,м для люды смс,1,м люд смс,м люды смс
4,4,м позн с д ж киев,1,м позн д ки,м позн д киев
...,...,...,...,...,...
78721,78721,у меня наверно говенные авиалинии nно вот все ...,2,наверн говен авиалин нно последн эт прост жалк...,наверно говенные авиалинии nно последние это п...
78722,78722,он по крайне мере бесплатный имеет нормальный ...,0,крайн мер бесплатн имеет нормальн формат файл ...,крайне мере бесплатный имеет нормальный формат...
78723,78723,я уточню у грефа ладно ладно,2,уточн греф ладн ладн,уточню грефа ладно ладно
78724,78724,да пока так и делаю но у ребят все быстро меня...,2,пок дела реб быстр меня,пока делаю ребят быстро меняется


# Лемматизация

In [78]:
lemm_t = []
for text in train['text_del']:
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text=' '.join(tokens)
        lemm_t.append(text)
    except Exception as e:
        print(e)
train['text_lemm']=lemm_t

In [79]:
train

,id,text,label,text_stem,text_del,text_lemm
0,0,я не знаю какая ты в жизни в постели в быту но...,1,зна жизн постел быт точн зна душ тво любл,знаю жизни постели быту точно знаю душу твою л...,знать жизнь постель быт точно знать душа твой ...
1,1,херсон рулит есть д для жизни я п,1,херсон рул д жизн п,херсон рулит д жизни п,херсон рулить д жизнь п \n
2,2,решил поглумиться итераций лр,2,реш поглум итерац лр,решил поглумиться итераций лр,решать поглумиться итерация лр \n
3,3,м для люды смс,1,м люд смс,м люды смс,м люда смс \n
4,4,м позн с д ж киев,1,м позн д ки,м позн д киев,м позн д киев \n
...,...,...,...,...,...,...
78721,78721,у меня наверно говенные авиалинии nно вот все ...,2,наверн говен авиалин нно последн эт прост жалк...,наверно говенные авиалинии nно последние это п...,наверно говенный авиалиния пно последний прост...
78722,78722,он по крайне мере бесплатный имеет нормальный ...,0,крайн мер бесплатн имеет нормальн формат файл ...,крайне мере бесплатный имеет нормальный формат...,крайне мера бесплатный иметь нормальный формат...
78723,78723,я уточню у грефа ладно ладно,2,уточн греф ладн ладн,уточню грефа ладно ладно,уточнять греф ладно ладно \n
78724,78724,да пока так и делаю но у ребят все быстро меня...,2,пок дела реб быстр меня,пока делаю ребят быстро меняется,пока делать ребята быстро меняться \n


In [80]:
def lemmatize_text(text):
    text_lem = mystem.lemmatize(text)
    tokens = [token for token in text_lem if token !=' ']
    return ' '.join(tokens)

In [81]:
train

,id,text,label,text_stem,text_del,text_lemm
0,0,я не знаю какая ты в жизни в постели в быту но...,1,зна жизн постел быт точн зна душ тво любл,знаю жизни постели быту точно знаю душу твою л...,знать жизнь постель быт точно знать душа твой ...
1,1,херсон рулит есть д для жизни я п,1,херсон рул д жизн п,херсон рулит д жизни п,херсон рулить д жизнь п \n
2,2,решил поглумиться итераций лр,2,реш поглум итерац лр,решил поглумиться итераций лр,решать поглумиться итерация лр \n
3,3,м для люды смс,1,м люд смс,м люды смс,м люда смс \n
4,4,м позн с д ж киев,1,м позн д ки,м позн д киев,м позн д киев \n
...,...,...,...,...,...,...
78721,78721,у меня наверно говенные авиалинии nно вот все ...,2,наверн говен авиалин нно последн эт прост жалк...,наверно говенные авиалинии nно последние это п...,наверно говенный авиалиния пно последний прост...
78722,78722,он по крайне мере бесплатный имеет нормальный ...,0,крайн мер бесплатн имеет нормальн формат файл ...,крайне мере бесплатный имеет нормальный формат...,крайне мера бесплатный иметь нормальный формат...
78723,78723,я уточню у грефа ладно ладно,2,уточн греф ладн ладн,уточню грефа ладно ладно,уточнять греф ладно ладно \n
78724,78724,да пока так и делаю но у ребят все быстро меня...,2,пок дела реб быстр меня,пока делаю ребят быстро меняется,пока делать ребята быстро меняться \n


In [82]:
train['text_lemm'][5]

'парень скучать девушка приселать прилольний ммс \n'

In [83]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#words = ['NLP', 'is', 'awesome']

label_encoder = LabelEncoder()
corpus_encoded = label_encoder.fit_transform(train['text_lemm'])
corpus_encoded

array([19840, 68792, 58634, ..., 67876, 50423, 67549])

In [84]:
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit_transform(corpus_encoded.reshape(-1, 1))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [85]:
train

,id,text,label,text_stem,text_del,text_lemm
0,0,я не знаю какая ты в жизни в постели в быту но...,1,зна жизн постел быт точн зна душ тво любл,знаю жизни постели быту точно знаю душу твою л...,знать жизнь постель быт точно знать душа твой ...
1,1,херсон рулит есть д для жизни я п,1,херсон рул д жизн п,херсон рулит д жизни п,херсон рулить д жизнь п \n
2,2,решил поглумиться итераций лр,2,реш поглум итерац лр,решил поглумиться итераций лр,решать поглумиться итерация лр \n
3,3,м для люды смс,1,м люд смс,м люды смс,м люда смс \n
4,4,м позн с д ж киев,1,м позн д ки,м позн д киев,м позн д киев \n
...,...,...,...,...,...,...
78721,78721,у меня наверно говенные авиалинии nно вот все ...,2,наверн говен авиалин нно последн эт прост жалк...,наверно говенные авиалинии nно последние это п...,наверно говенный авиалиния пно последний прост...
78722,78722,он по крайне мере бесплатный имеет нормальный ...,0,крайн мер бесплатн имеет нормальн формат файл ...,крайне мере бесплатный имеет нормальный формат...,крайне мера бесплатный иметь нормальный формат...
78723,78723,я уточню у грефа ладно ладно,2,уточн греф ладн ладн,уточню грефа ладно ладно,уточнять греф ладно ладно \n
78724,78724,да пока так и делаю но у ребят все быстро меня...,2,пок дела реб быстр меня,пока делаю ребят быстро меняется,пока делать ребята быстро меняться \n


# Анализ

In [86]:
X = train['text_lemm']
y = train['label']

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=42)
                                                    

In [88]:
mtags = train['label'].unique()
mtags

array([1, 2, 0])

# Naive Bayes Classifier

In [89]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [90]:
nb = Pipeline([('vect', CountVectorizer()),
              ('tfidf', TfidfTransformer()),
              ('clf', MultinomialNB()),]
)

In [91]:
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [92]:
from sklearn.metrics import classification_report
y_pred=nb.predict(X_test)

In [93]:
y_pred[1]

1

In [94]:
print(X_train[1], y_train[1], y_pred[1])

херсон рулить д жизнь п 
 1 1


In [95]:
from sklearn.metrics import accuracy_score
print('accuracy %s' %accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, labels=mtags))

accuracy 0.8667541705478872
              precision    recall  f1-score   support

           1       0.96      0.98      0.97     10486
           2       0.77      0.93      0.84      8632
           0       0.88      0.48      0.62      4500

    accuracy                           0.87     23618
   macro avg       0.87      0.80      0.81     23618
weighted avg       0.87      0.87      0.86     23618



# LogisticRegression

In [96]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
              ('tfidf', TfidfTransformer()),
              ('clf', LogisticRegression(n_jobs=1, C=1.0)),]
)

In [97]:
logreg.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1))])

In [98]:
y_pred = logreg.predict(X_test)

In [99]:
print('accuracy %s' %accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, labels=mtags))

accuracy 0.8779320856973495
              precision    recall  f1-score   support

           1       0.98      0.96      0.97     10486
           2       0.80      0.91      0.85      8632
           0       0.81      0.63      0.71      4500

    accuracy                           0.88     23618
   macro avg       0.86      0.83      0.84     23618
weighted avg       0.88      0.88      0.88     23618



# Baseline

In [100]:
features=['text_del']
target='label'

In [101]:
#train=train.set_index('id')
train_target=train[target]

In [102]:
X_train, X_test, y_train, y_test=train_test_split(train['text_del'], train['label'],
                                                  test_size=0.3, stratify=train_target, 
                                                  random_state=42)

In [103]:
y_train.value_counts(normalize=True)


1    0.446070
2    0.361908
0    0.192023
Name: label, dtype: float64

In [104]:
y_test.value_counts(normalize=True)

1    0.446100
2    0.361885
0    0.192015
Name: label, dtype: float64

In [105]:
model = Pipeline([('vect', CountVectorizer()),
              ('tfidf', TfidfTransformer()),
              ('clf', LogisticRegression(n_jobs=1, C=1.0)),]
)
model.fit(train['text_del'], train['label'])

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1))])

# Подготовка submission

В плане предобработки\генерации признаков делаем все тоже самое. Потом предсказываем моделью значения.

In [106]:
test['text'] = [multiple_spaces(numbers(punctuation(t_lower(text)))) for text in test['text']]

In [107]:
stemmed_t2 = []
for text in test['text']:
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = ' '. join(stemmed_tokens)
    stemmed_t2.append(text)

test['text_stem']=stemmed_t2

In [108]:
del_t2=[]
for text in test['text']:
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token !=' ']
    text = ' '.join(tokens)
    del_t2.append(text)
test['text_del']=del_t2

In [109]:
lemm_t2 = []
for text in test['text_del']:
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text=' '.join(tokens)
        lemm_t2.append(text)
    except Exception as e:
        print(e)
test['text_lemm']=lemm_t2

In [110]:
test['label']=model.predict(test['text_del'])

In [111]:
test

,id,text,text_stem,text_del,text_lemm,label
0,0,на зарешке сказано было байес хотя это и правд...,зарешк сказа байес хот эт правд bias,зарешке сказано байес хотя это правда bias,зарешка сказать байес хотя это правда bias \n,2
1,1,ну и зачем вы так сделали,сдела,сделали,сделать \n,2
2,2,комса в в каждой паре есть минимальная сумма о...,комс кажд пар минимальн сумм ордер,комса каждой паре минимальная сумма ордера,комс каждый пара минимальный сумма ордер \n,2
3,3,и в сериалах и в жизни,сериал жизн,сериалах жизни,сериал жизнь \n,1
4,4,пара м ж пригласят в гости стройную девушку се...,пар м приглас гост стройн девушк,пара м пригласят гости стройную девушку,пара м приглашать гость стройный девушка \n,1
...,...,...,...,...,...,...
33736,33736,веселая дев ищет парня для с о общения сначала...,весел дев ищет парн общен снача смс,веселая дев ищет парня общения сначала смс,веселый дева искать парень общение сначала смс \n,1
33737,33737,для своего логгера делай не для дефолтного,сво логгер дела дефолтн,своего логгера делай дефолтного,свой логгер делать дефолтный \n,0
33738,33738,для текстов подходит но фичи обычно всё же чем...,текст подход фич обычн все друг извлека,текстов подходит фичи обычно всё другим извлекают,текст подходить фич обычно извлекать \n,2
33739,33739,сообщество не развивается там где есть расстре...,сообществ развива расстрельн групп,сообщество развивается расстрельная группа,сообщество развиваться расстрельный группа \n,2


In [112]:
test[['id', 'label']].to_csv('sub_baseline.csv', index=None)

https://habr.com/ru/post/538458/